# Import

In [1]:
from utils import init_logger, timer, fix_seed
import random
import pandas as pd
pd.set_option('display.max_columns', 150)
import numpy as np
import category_encoders as ce
import matplotlib.pyplot as plt
import datetime as dt
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas(desc="Processing:")

fix_seed()
logger = init_logger()

In [2]:
PROJECT_NAME = "v3"

# Load

In [3]:
import pandas as pd
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

def load_data():
    df_train = pd.read_csv("../input/train.csv")
    df_test = pd.read_csv("../input/test.csv")
    df = pd.concat([df_train, df_test]).reset_index(drop=True)
    
    df['Fare'].fillna(df.query('Pclass==3 & Embarked=="S"')['Fare'].median(), inplace=True)
    
    titles = [
        "Mr.", "Miss.", "Mrs.", "Master.", "Dr.", "Rev.", "Col.", "Ms.", 
        "Mlle.", "Mme.", "Capt.", "Countess.", "Major.", "Jonkheer.", "Don.", 
        "Dona.", "Sir.", "Lady."
    ]
    df["Title"] = None
    for title in titles:
        df.loc[df["Name"].str.contains(title, regex=False), "Title"] = title
    df.loc[(df["Title"] != "Master.") & (df["Sex"] == "male"), "Title"] = "Mr."
    df.loc[(df["Title"] != "Mrs.") & (df["Sex"] == "female"), "Title"] = "Miss."
    
    df["Surname_Fare"] = df["Name"].map(lambda name: name.split(',')[0].strip()) + df["Fare"].astype(str)
    df["Count_Surname_Fare"] = df["Surname_Fare"].map(df["Surname_Fare"].value_counts())

    for title in df["Title"].unique():
        df.loc[df["Title"] == title, "Age"].fillna(df.loc[df["Title"] == title, "Age"].mean(), inplace=True)
    
    # ordinal encoding
    oe_columns = [
        "Sex",
        "Title",
        "Pclass",
        "Embarked"
    ]
    oe = ce.OrdinalEncoder(cols=oe_columns, handle_unknown='value')
    df = oe.fit_transform(df)

    
    # extract
    df.drop([
        "Name",
        "Ticket",
        "Cabin"
    ], axis=1, inplace=True)

    df_train, df_test = df[:len(df_train)], df[len(df_train):]
    
    # target encoding
    loo = ce.TargetEncoder(cols="Surname_Fare", handle_unknown="value")
    df_train = loo.fit_transform(df_train, df_train["Survived"])
    df_test = loo.transform(df_test)
    
    return df_train, df_test

In [4]:
with timer("read csv", logger):
    df_train, df_test = load_data()

2022/01/02 17:37:05 45 [INFO] [read csv] start.
2022/01/02 17:37:05 47 [INFO] [read csv] done in 0.028 seconds.


In [5]:
X_train = df_train.drop(["PassengerId", "Survived"], axis=1)
y_train = df_train["Survived"]
X_test = df_test.drop(["PassengerId", "Survived"], axis=1)

In [6]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Surname_Fare,Count_Surname_Fare
0,1,1,22.0,1,0,7.2500,1,1,0.383838,1
1,2,2,38.0,1,0,71.2833,2,2,0.383838,2
2,1,2,26.0,0,0,7.9250,1,3,0.383838,1
3,2,2,35.0,1,0,53.1000,1,2,0.468759,2
4,1,1,35.0,0,0,8.0500,1,1,0.383838,1


In [7]:
X_train.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title',
       'Surname_Fare', 'Count_Surname_Fare'],
      dtype='object')

# Training

In [8]:
from model_xgb import ModelXGB
from runner import Runner
from hyperopt import hp, fmin, tpe, space_eval

In [9]:
runner = Runner(PROJECT_NAME, ModelXGB, cv=False)

In [10]:
# space = {
#     "objective": "binary",
#     "metric": ["binary_logloss", "binary_error"],
#     "max_leaves": hp.uniformint("max_leaves", 30, 80),
#     "min_data_in_leaf": hp.uniformint("min_data_in_leaf", 10, 50),
#     "lambda_l1": hp.loguniform("lambda_l1", np.log(0.01), np.log(1)),
#     "lambda_l2": hp.loguniform("lambda_l2", np.log(0.01), np.log(1)),
#     "bagging_fraction": hp.uniform("bagging_fraction", 0.8, 1),
#     "feature_fraction": hp.uniform("feature_fraction", 0.8, 1),
#     "learning_rate": 0.01,
#     "seed": 0,
#     "verbose": -1,
# }

params = {
    'objective': 'binary:logistic',
    "learning_rate": 0.01,
    "subsample": 0.8,
    "lambda": 0.8,
    "alpha": 0.8,
}

train_params = {
    "num_boost_round": 5000,
    "early_stopping_rounds": 30,
    "verbose_eval": 100
}

In [10]:
def objective(args):
    logger.info(args)
    runner.train(X_train, y_train, args, train_params)
    return runner.get_score()

In [11]:
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100)
best_params = space_eval(space, best)

  0%|                                   | 0/100 [00:00<?, ?trial/s, best loss=?]

2022/01/01 18:13:35 2 [INFO] {'bagging_fraction': 0.8393722009273112, 'feature_fraction': 0.8910156806187918, 'lambda_l1': 0.017566243042311882, 'lambda_l2': 0.8430033228269593, 'learning_rate': 0.01, 'max_leaves': 44, 'metric': ('binary_logloss', 'binary_error'), 'min_data_in_leaf': 30, 'objective': 'binary', 'seed': 0, 'verbose': -1}


Training until validation scores don't improve for 40 rounds                    
  0%|                                   | 0/100 [00:00<?, ?trial/s, best loss=?]

/Users/g-ogaki/miniforge3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

/Users/g-ogaki/miniforge3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "



Early stopping, best iteration is:                                              
[101]	training's binary_logloss: 0.423016	training's binary_error: 0.151685	valid_1's binary_logloss: 0.425635	valid_1's binary_error: 0.128492
  1%|        | 1/100 [00:00<01:31,  1.08trial/s, best loss: 0.42563452297869253]

...

2022/01/01 18:15:05 2 [INFO] {'bagging_fraction': 0.897148850095819, 'feature_fraction': 0.9152163529496, 'lambda_l1': 0.21795723541697962, 'lambda_l2': 0.07551290201567866, 'learning_rate': 0.01, 'max_leaves': 37, 'metric': ('binary_logloss', 'binary_error'), 'min_data_in_leaf': 37, 'objective': 'binary', 'seed': 0, 'verbose': -1}


Training until validation scores don't improve for 40 rounds                    
 99%|███████▉| 99/100 [01:29<00:00,  1.31trial/s, best loss: 0.3604479323510402]

/Users/g-ogaki/miniforge3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

/Users/g-ogaki/miniforge3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "



Early stopping, best iteration is:                                              
[30]	training's binary_logloss: 0.551079	training's binary_error: 0.189607	valid_1's binary_logloss: 0.550284	valid_1's binary_error: 0.173184
100%|███████| 100/100 [01:30<00:00,  1.11trial/s, best loss: 0.3604479323510402]

2022/01/01 18:15:05 3 [INFO] best params: {'bagging_fraction': 0.8691099275893169, 'feature_fraction': 0.9433525231687435, 'lambda_l1': 0.057908447715479115, 'lambda_l2': 0.01474021398017653, 'learning_rate': 0.01, 'max_leaves': 48, 'metric': ('binary_logloss', 'binary_error'), 'min_data_in_leaf': 26, 'objective': 'binary', 'seed': 0, 'verbose': -1}


In [12]:
logger.info("best params: {}".format(best_params))

2022/01/01 18:15:41 1 [INFO] best params: {'bagging_fraction': 0.8691099275893169, 'feature_fraction': 0.9433525231687435, 'lambda_l1': 0.057908447715479115, 'lambda_l2': 0.01474021398017653, 'learning_rate': 0.01, 'max_leaves': 48, 'metric': ('binary_logloss', 'binary_error'), 'min_data_in_leaf': 26, 'objective': 'binary', 'seed': 0, 'verbose': -1}


# Inference

In [11]:
runner = Runner(PROJECT_NAME, ModelXGB, cv=True)
with timer("train", logger):
    runner.train(X_train, y_train, params, train_params)

2022/01/02 17:37:10 45 [INFO] [train] start.


[17:37:10] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426411619/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-logloss:0.68719	eval-logloss:0.68732
[100]	train-logloss:0.38564	eval-logloss:0.38989
[200]	train-logloss:0.29612	eval-logloss:0.32061
[300]	train-logloss:0.25847	eval-logloss:0.30303
[397]	train-logloss:0.23681	eval-logloss:0.30037
[17:37:12] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426411619/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-logloss:0.68699	eval-logloss:0.68727
[100]	train-logloss:0.38314	eval-logloss:0.40357
[200]	train-logloss:0.29661

2022/01/02 17:37:19 47 [INFO] [train] done in 8.721 seconds.


In [12]:
runner.get_score()

0.3282806

In [13]:
with timer("prediction", logger):
    prob = runner.predict(X_test)
pred = np.where(prob > 0.5, 1, 0)
pred

2022/01/02 17:37:21 45 [INFO] [prediction] start.
/Users/g-ogaki/miniforge3/lib/python3.9/site-packages/xgboost/core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(
2022/01/02 17:37:21 47 [INFO] [prediction] done in 0.030 seconds.


array([0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [14]:
submission = pd.DataFrame({
    "PassengerId": df_test["PassengerId"],
    "Survived": pred
})
submission
submission.to_csv(f"../submission/submission_{PROJECT_NAME}.csv", index=False)
submission

,PassengerId,Survived
891,892,0
892,893,0
893,894,0
894,895,0
895,896,1
...,...,...
1304,1305,0
1305,1306,1
1306,1307,0
1307,1308,0


In [15]:
!kaggle competitions submit titanic -f ../submission/submission_{PROJECT_NAME}.csv -m "loss: 0.5372970979437052"

100%|████████████████████████████████████████| 2.77k/2.77k [00:02<00:00, 957B/s]
Successfully submitted to Titanic - Machine Learning from Disaster